In [61]:
import pandas as pd
from datasets import load_dataset
import re
huggingface_ds = load_dataset("chillies/IELTS-writing-task-2-evaluation")['train']
huggingface_ds = pd.DataFrame(huggingface_ds)

In [62]:
evaluation = huggingface_ds["evaluation"]
evaluation.head()
Lexical_df = pd.DataFrame()
Lexical_df['prompt'] = huggingface_ds['prompt']
Lexical_df['essay'] = huggingface_ds['essay']
len(Lexical_df)

9833

In [63]:
def extract(text):
    start_keyword = "Lexical Resource"
    end_keyword = "Grammatical Range and Accuracy"
    start_index = text.find(start_keyword)
    end_index = text.find(end_keyword)
    if start_index != -1 and end_index != -1:
        return text[start_index + len(start_keyword):end_index].strip()
    else:
        return ""

def clean_text(text):
    cleaned_text = text.replace('*', '').replace('#', '').replace('\n', '').replace('-', '').replace('*', '').strip()
    cleaned_text = cleaned_text.strip(": -")
    cleaned_text = cleaned_text.replace("Suggested Band Score", "").strip()
    cleaned_text = cleaned_text.replace("(Lexical Resource)", "").strip()
    cleaned_text = cleaned_text.replace("(Vocabulary)", "").strip()
    cleaned_text = cleaned_text.strip(": -")
    return cleaned_text

In [64]:
Lexical_df['text'] = huggingface_ds["evaluation"].apply(extract)
Lexical_df['text'] = Lexical_df['text'].apply(clean_text)
Lexical_df['text'][10]

'4.0 The candidate demonstrates a good range of vocabulary, including some advanced vocabulary. The vocabulary is mostly accurate, although there are a few minor mistakes (e.g., "poeple" instead of "people," "mthod" instead of "method"). The vocabulary is appropriate for the context of the essay.'

In [65]:
Lexical_df['text'][20]

'The candidate\'s vocabulary is limited, with frequent repetitions and a lack of variety. There are several grammatical and spelling errors, such as "poeple," "mthod," and "employ."  : 2.0'

In [66]:
Lexical_df['text'][30]

'[6]The candidate has used a range of vocabulary throughout the essay, including some more complex and sophisticated words. However, there are some instances where the vocabulary could be more precise or varied. For example, the candidate could have used a more specific word than "reliable" in the introduction.'

In [67]:
empty_string_count = (Lexical_df['text'] == '').sum()
print(f"Number of rows with empty '': {empty_string_count}")

Number of rows with empty '': 4


In [68]:
Lexical_df["score"] = Lexical_df['text'].str.extract(r'(\d+\.?\d*)').astype(float)
Lexical_df["score"]

0       7.0
1       4.0
2       6.0
3       5.0
4       3.0
       ... 
9828    6.0
9829    4.0
9830    7.0
9831    7.5
9832    6.0
Name: score, Length: 9833, dtype: float64

In [69]:
nan_count = Lexical_df['score'].isna().sum()
nan_count

74

In [70]:
def clean_text_v2(text):
    cleaned_text = text.replace('*', '').replace('#', '').replace('\n', '').replace('-', '').replace('*', '').strip()
    cleaned_text = re.sub(r"\d+\.*\d*", "", cleaned_text).strip()
    cleaned_text = re.sub(r"\[\d+\.*\d*\]", "", cleaned_text).strip()
    cleaned_text = cleaned_text.strip(": -")
    cleaned_text = cleaned_text.replace("Suggested Band Score", "").strip()
    cleaned_text = cleaned_text.replace("(Lexical Resource)", "").strip()
    cleaned_text = cleaned_text.replace("[]", "").strip()
    return cleaned_text

In [71]:
Lexical_df['text'] = Lexical_df['text'].apply(clean_text_v2)

In [72]:
Lexical_df['text'][10]

'The candidate demonstrates a good range of vocabulary, including some advanced vocabulary. The vocabulary is mostly accurate, although there are a few minor mistakes (e.g., "poeple" instead of "people," "mthod" instead of "method"). The vocabulary is appropriate for the context of the essay.'

In [73]:
Lexical_df['text'][20]

'The candidate\'s vocabulary is limited, with frequent repetitions and a lack of variety. There are several grammatical and spelling errors, such as "poeple," "mthod," and "employ."'

In [74]:
Lexical_df['text'][30]

'The candidate has used a range of vocabulary throughout the essay, including some more complex and sophisticated words. However, there are some instances where the vocabulary could be more precise or varied. For example, the candidate could have used a more specific word than "reliable" in the introduction.'

In [75]:
Lexical_df = Lexical_df[['prompt', 'essay', 'text', 'score']]
Lexical_df

,prompt,essay,text,score
0,Interviews form the basic criteria for most la...,It is believed by some experts that the tradit...,The candidate demonstrates a good range of voc...,7.0
1,Interviews form the basic selecting criteria f...,Nowadays numerous huge firms allocate an inter...,The candidate has used a range of vocabulary t...,4.0
2,Interview form the basic selection criteria fo...,The interview section is the most vital part o...,The candidate demonstrates a good range of voc...,6.0
3,Interviews form the basic selection criteria f...,It is argued that the best method to recruit e...,The candidate has used a limited range of voca...,5.0
4,Interviews from the basic selecting criteria f...,Nowadays many companies conduct interviews bef...,The vocabulary used is limited and repetitive....,3.0
...,...,...,...,...
9828,Nations should spend more money on skills and ...,"These days, many countries use a huge amount o...",The essay demonstrates a limited range of voca...,6.0
9829,Nations should spend more money on skills and ...,Skills are required in order to achieve succes...,The essay exhibits a limited range of vocabula...,4.0
9830,Nations should spend more money on skills and ...,There is a contradictory view among the people...,"The essay exhibits a good range of vocabulary,...",7.0
9831,Nations should spend more money on skills and ...,Many today feel that countries should prioriti...,The essay demonstrates a good range of vocabul...,7.5


In [76]:
empty_string_count = (Lexical_df['text'] == '').sum()
print(f"Number of rows with empty 'text': {empty_string_count}")
Lexical_df = Lexical_df[Lexical_df['text'] != '']

Number of rows with empty 'text': 4


In [77]:
nan_count = Lexical_df.isna().any(axis=1).sum()
nan_count

70

In [78]:
Lexical_df = Lexical_df.dropna(subset=['score'])
nan_count = Lexical_df.isna().any(axis=1).sum()
nan_count

0

In [79]:
len(Lexical_df)

9759

In [80]:
csv_path = 'data/Lexical.csv'
Lexical_df.to_csv(csv_path, index=False)